In [1]:
!pip install pytorch-pretrained-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
%cd /content/drive/MyDrive/banner-master

/content/drive/.shortcut-targets-by-id/1-_YhZjR5RRUfyhjAlT-wbLioqxzpdA46/banner-master


In [5]:
!ls

cost.py  dataset.py    methodology.jpg	pred_labels_banner.txt	README.md
crf.py	 inference.py  models		pred_labels.txt		run.py
data	 main.py       net.py		__pycache__		trainer.py


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
import torch.nn as nn
import json
from trainer import train,eval
from cost import crit_weights_gen
from net import Net
from dataset import NerDataset, pad, VOCAB, tag2idx, idx2tag
import dataset
import torch.optim as optim
import os

# dataset.VOCAB = ('<PAD>', 'O',
# 'I-GRP',
# 'B-PROD',
# 'I-PER',
# 'I-CW',
# 'I-CORP',
# 'B-PER',
# 'B-CORP',
# 'B-GRP',
# 'B-LOC',
# 'B-CW',
# 'I-PROD',
# 'I-LOC')
#
# dataset.tag2idx = {tag: idx for idx, tag in enumerate(VOCAB)}
# dataset.idx2tag = {idx: tag for idx, tag in enumerate(VOCAB)}

print(dataset.VOCAB)

batch_size = 32
lr = 0.001
n_epochs = 20
finetuning = True
top_rnns = True
trainset = "data/train.jsonl"
testset = "data/test.jsonl"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Net(top_rnns, len(VOCAB), device, finetuning)
#model.load_state_dict(torch.load('models/banner_model.pt'))
model.to(device)

# with open(trainset) as infile:
#     data = json.load(infile)

data = []
for line in open(trainset, 'r', encoding="utf8"):
    data.append(json.loads(line))

new = data
train_texts, train_labels = list(zip(*map(lambda d: (d['tokens'], d['tags']), new)))

sents_train, tags_li_train = [], []
for x in train_texts:
    sents_train.append(["[CLS]"] + x + ["[SEP]"])
for y in train_labels:
    tags_li_train.append(["<PAD>"] + y + ["<PAD>"])

train_dataset = NerDataset(sents_train, tags_li_train)

train_iter = torch.utils.data.DataLoader(dataset=train_dataset,
                             batch_size= batch_size,
                             shuffle=True,
                             collate_fn=pad,
                             num_workers=0
                             )

optimizer = optim.Adam(model.parameters(), lr = lr)
# data_dist = [7237, 15684, 714867, 759, 20815, 9662, 8512, 37529, 70025]
# crit_weights = crit_weights_gen(0.5,0.9,data_dist)
#insert 0 cost for ignoring <PAD>
# crit_weights.insert(0,0)
# crit_weights = torch.tensor(crit_weights).to(device)
criterion = nn.CrossEntropyLoss()



100%|██████████| 995526/995526 [00:00<00:00, 4207706.60B/s]


('<PAD>', 'O', 'I-GRP', 'B-PROD', 'I-PER', 'I-CW', 'I-CORP', 'B-PER', 'B-CORP', 'B-GRP', 'B-LOC', 'B-CW', 'I-PROD', 'I-LOC')


100%|██████████| 662804195/662804195 [00:10<00:00, 60501339.07B/s]


In [7]:
device

'cuda'

In [8]:
data = []
for line in open(testset, 'r', encoding="utf8"):
    data.append(json.loads(line))

new = data
test_texts, test_labels = list(zip(*map(lambda d: (d['tokens'], d['tags']), new)))

sents_test, tags_li_test = [], []
for x in test_texts:
    sents_test.append(["[CLS]"] + x + ["[SEP]"])
for y in test_labels:
    tags_li_test.append(["<PAD>"] + y + ["<PAD>"])

test_dataset = NerDataset(sents_test, tags_li_test)

test_iter = torch.utils.data.DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             collate_fn = pad,
                             num_workers=0
                             )

In [ ]:
for epoch in range(1, n_epochs+1):
    if epoch>10:
        optimizer = optim.Adam([
                                {"params": model.fc.parameters(), "lr": 0.0005},
                                {"params": model.bert.parameters(), "lr": 5e-5},
                                {"params": model.rnn.parameters(), "lr": 0.0005},
                                {"params": model.crf.parameters(), "lr": 0.0005}
                                ],)
    train(model, train_iter, optimizer, criterion, epoch)
    # pred = eval(model, test_iter, epoch)

    fname = os.path.join("models", str(epoch))
    torch.save(model.state_dict(), f"{fname}.pt")


# pred = eval(model, test_iter)
# for x in range(len(pred[0])):
#     if pred[0][x] == '<PAD>':
#         pred[0][x] = 'O'

# all_preds = sents_test[0][1:-1],pred[0][1:-1]

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Net(top_rnns, len(VOCAB), device, finetuning)
model.load_state_dict(torch.load('models/20.pt'))
model.to(device)

In [19]:
pred = eval(model, test_iter)

In [ ]:
print(pred)

In [21]:
all_preds = []
for x in range(len(pred)):
    if pred[x][0] == '<PAD>':
      pred[x][0] = 'O'
    # all_preds.append((sents_test[x][1:-1],pred[x][1:-1]))
    all_preds.append(pred[x][1:-1])

In [ ]:
all_preds

In [13]:
with open('pred_labels_banner.txt', 'w') as f:
  for sentence_pred in all_preds:
    for pred in sentence_pred:
      f.write(pred+'\n')
    f.write('\n')

In [ ]:
!ls

cost.py  dataset.py    methodology.jpg	pred_labels_banner.txt	README.md
crf.py	 inference.py  models		pred_labels.txt		run.py
data	 main.py       net.py		__pycache__		trainer.py


In [ ]:
!ls

cost.py  dataset.py    methodology.jpg	pred_labels_banner.txt	README.md
crf.py	 inference.py  models		pred_labels.txt		run.py
data	 main.py       net.py		__pycache__		trainer.py
